In [40]:
# Imports
import json
import logging
import os

import numpy as np
from hapi import absorptionCoefficient_Voigt, absorptionCoefficient_Lorentz, fetch_by_ids

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
dirname = "."


def hapi_setup(
    table_name,
    sat_name,
    band,
    DOWNLOAD_FILE_PATH,
    iso_id_file=os.path.join(dirname, "iso_ids.json"),
    sat_param_file=os.path.join(dirname, "satellite_params.json"),
    atm_params_file=os.path.join(dirname, "atm_params.json"),
    parameter_groups=["160-char"],
    parameters=None,
):
    """
    Downloads  line-by-line data from the HITRANonline site to a specified location
    and Calculate absorption coefficients at different pressures and temperatures.

    Arguments
    table_name          [String]    :    name of table to fetch in (name of the isotope eg H20)
    sat_name            [String]    :    name of the satellite (eg Sentinel2A, Sentinel2B)
    band                [String]    :    name of band (eg B11, B12)
    DOWNLOAD_FILE_PATH  [String]    :    file path to download to (must end in / ))
    iso_id_file         [String]    :    config file of list of isotopologue id's. Default : iso_ids.json
    sat_param_file      [String]    :    config file contaning different numbermin and numbermax wavelengths for bands on different satellites. Default: satellite_params.json
    atm_params_file     [String]    :    config file with different atmospheric variables
    parameters          [String]    :    specifiy extra parameters  explicitly, eg 'deltap air'
    """
    # get params from config JSON files
    sat_params = json.load(open(sat_param_file))
    # global isotropologue IDs from https://hitran.org/docs/iso-meta/
    iso_ids = json.load(open(iso_id_file))
    atm_params = json.load(open(atm_params_file))
    download_path = DOWNLOAD_FILE_PATH + sat_name + "/" + band + "/"

    # Create directory if it does not exist
    if not os.path.exists(os.path.dirname(download_path)):
        os.makedirs(os.path.dirname(download_path))

    # change to directory to download into
    # os.chdir(download_path)

    # Read min/max wavelengths of satellite specific bands
    wave_min_nm = sat_params[sat_name][band]["nmmin"]
    wave_max_nm = sat_params[sat_name][band]["nmmax"]

    # Convert to Wavelengths to Wavenumbers, include in sat_params
    wave_min_number, wave_max_number = (1e7 / wave_max_nm, 1e7 / wave_min_nm)

    # Define Wavestep Number
    # TODO:Understand assumptions behind that number, put into sat_params
    wave_step_nm = atm_params["wave_step_nm"]
    min_p = atm_params["min_p"]
    max_p = atm_params["max_p"]
    delta_p = atm_params["delta_p"]
    min_temp = atm_params["min_temp"]
    max_temp = atm_params["max_temp"]
    delta_temp = atm_params["delta_temp"]
    number_step = atm_params["number_step"]

    p_vector = np.arange(min_p, max_p + delta_p, delta_p)
    temp_vector = np.arange(min_temp, max_temp + delta_temp, delta_temp)

    len_press = len(p_vector)
    len_temp = len(temp_vector)

    # Wavelength range
    wavelength = np.arange(wave_min_nm, wave_max_nm, wave_step_nm)
    len_wave = len(wavelength)

    absorption_arr = np.zeros((len_press, len_temp, len_wave))

    logger.info(f"table_name: {table_name}")
    logger.info(f"iso_ids: {iso_ids[table_name]}")
    logger.info(f"wave_min_number: {wave_min_number} cm⁻¹")
    logger.info(f"wave_max_number: {wave_max_number}")
    logger.info("Downloading HAPI data with `fetch_by_ids`")

    # Case if parameters argument is/isnt given
    if parameters is None:
        fetch_by_ids(
            table_name,
            iso_ids[table_name],
            wave_min_number,
            wave_max_number,
            ParameterGroups=parameter_groups,
        )

    else:
        fetch_by_ids(
            table_name,
            iso_ids[table_name],
            wave_min_number,
            wave_max_number,
            ParameterGroups=parameter_groups,
            Parameters=parameters,
        )

    logger.info(f"Calculating Atmospheric cross sections for {table_name}")

    logger.info(f"Temperatures: {temp_vector}")
    logger.info(f"Pressures: {p_vector}")
    # calculate atmospheric cross sections across pressure and temperature vectors
    for i, p_tmp in enumerate(p_vector):
        for j, t_tmp in enumerate(temp_vector):

            # Calculate wavenumber and absorption coefficient
            logger.info(f"Getting absorption coefficients for temperature {t_tmp} and pressure {p_tmp}")
            logger.info(f"Wave number step: {number_step}")
            wavenumber_temp, absorption_temp = absorptionCoefficient_Voigt(
                SourceTables=table_name,
                Environment={"T": t_tmp, "p": p_tmp},
                WavenumberStep=number_step,
            )

            # Convert to wavelength
            wavelength_temp = 1e7 / wavenumber_temp

            # Interpolate onto common grid
            absorption_temp_interp = np.interp(wavelength, wavelength_temp[::-1], absorption_temp[::-1])

            # Append to liist
            absorption_arr[i, j] = absorption_temp_interp

    # Save arrays
    np.save(os.path.join(download_path, "abs_wave_hapi_" + sat_name + "_band_" + band + ".npy"), wavelength)
    np.save(os.path.join(download_path, "abs_press_hapi_" + sat_name + "_band_" + band + ".npy"), p_vector)
    np.save(os.path.join(download_path, "abs_temp_hapi_" + sat_name + "_band_" + band + ".npy"), temp_vector)
    np.save(
        os.path.join(download_path, 
        "abs_" + table_name + "_hapi_" + sat_name + "_band_" + band + ".npy"),
        absorption_arr,
    )
    # os.chdir("../../../..")

In [41]:
# os.chdir('../../../..')
# hapi_setup("CH4", "EMIT", "VSWIR", "data/hapi_data/")
# hapi_setup("H20", "EMIT", "VSWIR", "data/hapi_data/")
hapi_setup("CO2", "EMIT", "VSWIR", "data/hapi_data/")

INFO:root:table_name: CO2
INFO:root:iso_ids: [7, 8, 9]
INFO:root:wave_min_number: 4000.0 cm⁻¹
INFO:root:wave_max_number: 27027.027027027027
INFO:root:Downloading HAPI data with `fetch_by_ids`



Data is fetched from http://hitran.org

BEGIN DOWNLOAD: CO2
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  65536 bytes written to ./CO2.data
  6

INFO:root:Calculating Atmospheric cross sections for CO2
INFO:root:Temperatures: [250 275 300 325]
INFO:root:Pressures: [0.8 0.9 1.  1.1 1.2]
INFO:root:Getting absorption coefficients for temperature 250 and pressure 0.8
INFO:root:Wave number step: 0.01


                     Lines parsed: 127713
PROCESSED
{'air': 1.0}


INFO:root:Getting absorption coefficients for temperature 275 and pressure 0.8
INFO:root:Wave number step: 0.01


...

{'air': 1.0}


INFO:root:Getting absorption coefficients for temperature 300 and pressure 1.2
INFO:root:Wave number step: 0.01


{'air': 1.0}


INFO:root:Getting absorption coefficients for temperature 325 and pressure 1.2
INFO:root:Wave number step: 0.01


{'air': 1.0}
